In [ ]:

def get_data(loc):
    import requests
    import datetime
    import pvlib
    try:
        try:
            lat = result.Latitude[result["Location"] == loc].item()
            lon = result.Longitude[result["Location"] == loc].item()
        except:
            print("loc not found")
        # --------------------------
        # 2) Current time (UTC, rounded to nearest hour)
        # --------------------------
        now = datetime.datetime.now(datetime.UTC).replace(minute=0, second=0, microsecond=0)
        time_str = now.strftime("%Y-%m-%dT%H:00")  # format like "2025-09-12T08:00" .

        # --------------------------
        # 3) Get weather forecast from API
        # --------------------------
        url = f"https://api.open-meteo.com/v1/forecast?latitude={lat}&longitude={lon}" \
              "&hourly=temperature_2m,windspeed_10m,direct_radiation,diffuse_radiation"
        data = requests.get(url).json()  # request + convert to dictionary

        # If no hourly data, stop
        if "hourly" not in data:
            raise RuntimeError(f"No hourly data! Reason: {data.get('reason', 'Unknown')}")

        hourly = data["hourly"]

        # --------------------------
        # 4) Extract values for current hour
        # --------------------------
        if time_str not in hourly["time"]:
            raise RuntimeError(f"Time {time_str} not found in API data!")

        i = hourly["time"].index(time_str)  # find index of current hour

        dni = float(hourly["direct_radiation"][i])  # Direct normal irradiance
        dhi = float(hourly["diffuse_radiation"][i])  # Diffuse horizontal irradiance
        ghi = dni + dhi  # Approximate global irradiance
        temp_air = float(hourly["temperature_2m"][i])  # Air temp (°C)
        wind_speed = float(hourly["windspeed_10m"][i])  # Wind speed (m/s)

        # --------------------------
        # 5) Solar position
        # --------------------------
        solpos = pvlib.solarposition.get_solarposition(now, lat, lon)
        solar_elev = float(solpos["elevation"].iloc[0])  # Sun height in degrees

        # --------------------------
        # 6) Plane-of-array irradiance (for tilted panel)
        # --------------------------
        tilt, azim = 30, 180  # 30° tilt, facing south
        poa = pvlib.irradiance.get_total_irradiance(
            surface_tilt=tilt,
            surface_azimuth=azim,
            solar_zenith=90 - solar_elev,  # zenith = 90 - elevation
            solar_azimuth=solpos["azimuth"].iloc[0],
            dni=dni, ghi=ghi, dhi=dhi
        )

        # Break down POA (plane-of-array) components
        poa_direct = float(poa["poa_direct"])
        poa_sky_diffuse = float(poa["poa_sky_diffuse"])

        # --------------------------
        # 7) Collect results (no Int, no poa_ground_diffuse)
        # --------------------------
        print("Current compatible data entry:")
        print(f"poa_direct: {poa_direct}")
        print(f"poa_sky_diffuse: {poa_sky_diffuse}")
        print(f"solar_elevation: {solar_elev}")
        print(f"temp_air: {temp_air}")
        print(f"wind_speed: {wind_speed}")

        # --------------------------
        # 8) Return as variables (not dictionary)
        # --------------------------
        return poa_direct, poa_sky_diffuse, solar_elev, temp_air, wind_speed
    except:
        return 10, 10, 10, 20, 10

In [ ]:
time_stamp = datetime.datetime.now().month

In [5]:
def pridictionn(loc):
    import requests
    import datetime
    import pvlib
    import pandas as pd
    try:
        result=pd.read_csv("results.csv")
        try:
            lat = result.Latitude[result["Location"] == loc].item()
            lon = result.Longitude[result["Location"] == loc].item()
        except:
            print("loc not found")
        # --------------------------
        # 2) Current time (UTC, rounded to nearest hour)
        # --------------------------
        now = datetime.datetime.now(datetime.UTC).replace(minute=0, second=0, microsecond=0)
        time_str = now.strftime("%Y-%m-%dT%H:00")  # format like "2025-09-12T08:00" .

        # --------------------------
        # 3) Get weather forecast from API
        # --------------------------
        url = f"https://api.open-meteo.com/v1/forecast?latitude={lat}&longitude={lon}" \
              "&hourly=temperature_2m,windspeed_10m,direct_radiation,diffuse_radiation"
        data = requests.get(url).json()  # request + convert to dictionary

        # If no hourly data, stop
        if "hourly" not in data:
            raise RuntimeError(f"No hourly data! Reason: {data.get('reason', 'Unknown')}")

        hourly = data["hourly"]

        # --------------------------
        # 4) Extract values for current hour
        # --------------------------
        if time_str not in hourly["time"]:
            raise RuntimeError(f"Time {time_str} not found in API data!")

        i = hourly["time"].index(time_str)  # find index of current hour

        dni = float(hourly["direct_radiation"][i])  # Direct normal irradiance
        dhi = float(hourly["diffuse_radiation"][i])  # Diffuse horizontal irradiance
        ghi = dni + dhi  # Approximate global irradiance
        temp_air = float(hourly["temperature_2m"][i])  # Air temp (°C)
        wind_speed = float(hourly["windspeed_10m"][i])  # Wind speed (m/s)

        # --------------------------
        # 5) Solar position
        # --------------------------
        solpos = pvlib.solarposition.get_solarposition(now, lat, lon)
        solar_elev = float(solpos["elevation"].iloc[0])  # Sun height in degrees

        # --------------------------
        # 6) Plane-of-array irradiance (for tilted panel)
        # --------------------------
        tilt, azim = 30, 180  # 30° tilt, facing south
        poa = pvlib.irradiance.get_total_irradiance(
            surface_tilt=tilt,
            surface_azimuth=azim,
            solar_zenith=90 - solar_elev,  # zenith = 90 - elevation
            solar_azimuth=solpos["azimuth"].iloc[0],
            dni=dni, ghi=ghi, dhi=dhi
        )

        # Break down POA (plane-of-array) components
        poa_direct = float(poa["poa_direct"])
        poa_sky_diffuse = float(poa["poa_sky_diffuse"])

        # --------------------------
        # 7) Collect results (no Int, no poa_ground_diffuse)
        # --------------------------
        print("Current compatible data entry:")
        print(f"poa_direct: {poa_direct}")
        print(f"poa_sky_diffuse: {poa_sky_diffuse}")
        print(f"solar_elevation: {solar_elev}")
        print(f"temp_air: {temp_air}")
        print(f"wind_speed: {wind_speed}")

        # --------------------------
        # 8) Return as variables (not dictionary)
        # --------------------------
        #return poa_direct, poa_sky_diffuse, solar_elev, temp_air, wind_speed
    #except:
        #return 10, 10, 10, 20, 10
        
        def prid(model_path, loc, time_stamp, poa_direct, poa_sky_diffuse, solar_elev, wind_speed, temp_air):
            import numpy as np
            from joblib import load
            import json
        
            # Load model
            model = load(model_path)
        
            # Load column names (all 246 column names)
            with open("column_names.json", "r") as f:
                column_names = json.load(f)  # list of strings, length 246
        
            location_names = [name.lower() for name in column_names[:240]]
            loc_lower = loc.lower()
        
            xi = np.zeros(246)
        
            if loc_lower != "afzalpur":
                if loc_lower not in location_names:
                    raise ValueError(f"Location '{loc}' not found in location list.")
                loc_index = location_names.index(loc_lower)
                xi[loc_index] = 1
            # if loc is afzalpur, all zeros in dummies by default (xi initialized to zeros)
        
            # Set numeric features
            xi[240] = time_stamp
            xi[241] = poa_direct
            xi[242] = poa_sky_diffuse
            xi[243] = solar_elev
            xi[244] = wind_speed
            xi[245] = temp_air
        
            xi = xi.reshape(1, -1)
        
            prediction = model.predict(xi)
            if prediction<0:
                prediction=0
            return prediction
    
        time_stamp = datetime.datetime.now().month
    
        ans=prid("Linear_Regression.pkl", loc, time_stamp, poa_direct, poa_sky_diffuse, solar_elev, temp_air, wind_speed)
        return ans.item()
        
    except Exception as e:
            print(e)


In [21]:
result=pridictionn("chitradurga")

loc not found
cannot access local variable 'lat' where it is not associated with a value


In [7]:
result

601.1812563786077

In [17]:
def pridictionn2(loc):
    import requests
    import datetime
    import pvlib
    import pandas as pd
    try:
        result=pd.read_csv("results.csv")
        try:
            lat = result.Latitude[result["Location"] == loc].item()
            lon = result.Longitude[result["Location"] == loc].item()
        except:
            try:
                from geopy.geocoders import Nominatim

                # Create a geolocator object
                geolocator = Nominatim(user_agent="geoapi")
                
                # Get location details
                location = geolocator.geocode(loc)
                
                # Extract latitude and longitude
                lat, lon = location.latitude, location.longitude
            except:
                print("loc not found")
        # --------------------------
        # 2) Current time (UTC, rounded to nearest hour)
        # --------------------------
        now = datetime.datetime.now(datetime.UTC).replace(minute=0, second=0, microsecond=0)
        time_str = now.strftime("%Y-%m-%dT%H:00")  # format like "2025-09-12T08:00" .

        # --------------------------
        # 3) Get weather forecast from API
        # --------------------------
        url = f"https://api.open-meteo.com/v1/forecast?latitude={lat}&longitude={lon}" \
              "&hourly=temperature_2m,windspeed_10m,direct_radiation,diffuse_radiation"
        data = requests.get(url).json()  # request + convert to dictionary

        # If no hourly data, stop
        if "hourly" not in data:
            raise RuntimeError(f"No hourly data! Reason: {data.get('reason', 'Unknown')}")

        hourly = data["hourly"]

        # --------------------------
        # 4) Extract values for current hour
        # --------------------------
        if time_str not in hourly["time"]:
            raise RuntimeError(f"Time {time_str} not found in API data!")

        i = hourly["time"].index(time_str)  # find index of current hour

        dni = float(hourly["direct_radiation"][i])  # Direct normal irradiance
        dhi = float(hourly["diffuse_radiation"][i])  # Diffuse horizontal irradiance
        ghi = dni + dhi  # Approximate global irradiance
        temp_air = float(hourly["temperature_2m"][i])  # Air temp (°C)
        wind_speed = float(hourly["windspeed_10m"][i])  # Wind speed (m/s)

        # --------------------------
        # 5) Solar position
        # --------------------------
        solpos = pvlib.solarposition.get_solarposition(now, lat, lon)
        solar_elev = float(solpos["elevation"].iloc[0])  # Sun height in degrees

        # --------------------------
        # 6) Plane-of-array irradiance (for tilted panel)
        # --------------------------
        tilt, azim = 30, 180  # 30° tilt, facing south
        poa = pvlib.irradiance.get_total_irradiance(
            surface_tilt=tilt,
            surface_azimuth=azim,
            solar_zenith=90 - solar_elev,  # zenith = 90 - elevation
            solar_azimuth=solpos["azimuth"].iloc[0],
            dni=dni, ghi=ghi, dhi=dhi
        )

        # Break down POA (plane-of-array) components
        poa_direct = float(poa["poa_direct"])
        poa_sky_diffuse = float(poa["poa_sky_diffuse"])

        # --------------------------
        # 7) Collect results (no Int, no poa_ground_diffuse)
        # --------------------------
        print("Current compatible data entry:")
        print(f"poa_direct: {poa_direct}")
        print(f"poa_sky_diffuse: {poa_sky_diffuse}")
        print(f"solar_elevation: {solar_elev}")
        print(f"temp_air: {temp_air}")
        print(f"wind_speed: {wind_speed}")

        # --------------------------
        # 8) Return as variables (not dictionary)
        # --------------------------
        #return poa_direct, poa_sky_diffuse, solar_elev, temp_air, wind_speed
    #except:
        #return 10, 10, 10, 20, 10
        
        def prid(model_path, loc, time_stamp, poa_direct, poa_sky_diffuse, solar_elev, wind_speed, temp_air):
            import numpy as np
            from joblib import load
            import json
        
            # Load model
            model = load(model_path)
        
            # Load column names (all 246 column names)
            with open("column_names.json", "r") as f:
                column_names = json.load(f)  # list of strings, length 246
        
            location_names = [name.lower() for name in column_names[:240]]
            loc_lower = loc.lower()
        
            xi = np.zeros(246)
        
            # Set numeric features
            xi[240] = time_stamp
            xi[241] = poa_direct
            xi[242] = poa_sky_diffuse
            xi[243] = solar_elev
            xi[244] = wind_speed
            xi[245] = temp_air

            if loc_lower != "afzalpur":
                try:
                    loc_index = location_names.index(loc_lower)
                    xi[loc_index] = 1
                except :
                    print("globel model aactivate")
                
            #if loc_lower not in location_names:
                    #print(f"Location '{loc}' not found in location list.")
            # if loc is afzalpur, all zeros in dummies by default (xi initialized to zeros)
        
            
            xi = xi.reshape(1, -1)
            
            prediction = model.predict(xi)

            if prediction<0:
                prediction=0
            return prediction
    
        time_stamp = datetime.datetime.now().month
    
        ans=prid("Linear_Regression.pkl", loc, time_stamp, poa_direct, poa_sky_diffuse, solar_elev, temp_air, wind_speed)
        return ans.item()
        
    except Exception as e:
            print(e)


In [32]:
result=pridictionn2("Gadag")

Current compatible data entry:
poa_direct: 386.5967239763636
poa_sky_diffuse: 226.72208655980933
solar_elevation: 49.085646269896976
temp_air: 28.0
wind_speed: 11.9


d:\Projects\PV\venv\Lib\site-packages\sklearn\utils\validation.py:2749: UserWarning: X does not have valid feature names, but LinearRegression was fitted with feature names
  warnings.warn(


In [33]:
result

381.63676182230586

In [31]:
result=pridictionn2("Bengaluru")

Current compatible data entry:
poa_direct: 671.8184157865423
poa_sky_diffuse: 127.82274015923406
solar_elevation: 50.36735276148716
temp_air: 25.8
wind_speed: 12.7


d:\Projects\PV\venv\Lib\site-packages\sklearn\utils\validation.py:2749: UserWarning: X does not have valid feature names, but LinearRegression was fitted with feature names
  warnings.warn(
